In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import SVC

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from sklearn.decomposition import PCA

import joblib

In [ ]:
#import file and reading few lines
# dataTrain = pd.read_csv('FeaturesWhite.csv')
# dataEval = pd.read_csv('EvalWithWhite.csv')
# dataTrain = pd.read_csv('FeaturesNoWhite.csv')
# dataEval = pd.read_csv('EvalWithNoWhite.csv')
# dataTrain = pd.read_csv('HistogramWhite.csv')
# dataEval = pd.read_csv('EvalHistogramWhite.csv')
dataTrain = pd.read_csv('ProfileWhite.csv')
dataEval = pd.read_csv('EvalProfileWhite.csv')

dataTrain = dataTrain.drop("image", axis = 1)
dataEval = dataEval.drop("image", axis = 1)

In [ ]:
def ChangeLabel(dataToEdit , listOfLabels1, label1, listOfLabels2, label2):
  data = pd.DataFrame()
  data = data.append(dataToEdit)
  for label in listOfLabels1:
    data["label"].replace({label: label1,}, inplace=True)
  for label in listOfLabels2:
    data["label"].replace({label: label2}, inplace=True)
  
  ndata = data[data["label"] == label1]
  ndata = ndata.append(data[data["label"] == label2])

  return ndata

MODEL

In [ ]:
#SVM
def SVM(X_train, y_train, name):
  hyper_param = {'C': [0.1, 1, 10, 100], 
                'gamma': [1, 0.1, 0.01, 0.001], 
                'kernel': ['linear', 'rbf', 'poly']}
  model =SVC(probability = True)
  gs = GridSearchCV(estimator=model,
                      param_grid=hyper_param,
                      cv=StratifiedKFold(5).split(X_train, y_train),
                      scoring='accuracy',
                      refit=True,
                      verbose=3)

  gs.fit(X_train, y_train)

  joblib.dump(gs.best_estimator_, f'{name}.npy')

  return gs.best_estimator_, gs.best_params_, gs.best_score_

STAGE 1

In [ ]:
# L1Stage1=['1', 'I', 'l', 'i', 't','j', 'T', 'J', 'Z', 'z',
#           '7', 'E', 'F', 'H', 'L', 'W', 'w', 'V', 'v', 
#           'X', 'x', 'Y', 'y', 'P', 'p', 'A', 'f', 'K', 'R', 'k','U']

# L2Stage1=['6', 'a', 'e', 'r', '5', 'S', 's', '2', 'M', 'N','m', 
#           'n', 'h', 'b', 'u', '0', 'O', 'o', 'D', 'Q', 
#           'C', 'G', 'c', '3', 'B', '8','d', '9', 'g', 'q', '4']

# L1Stage1=['T', 'I', 'l', 'i', 't','j', '1', 'J', 'f', '9', 'g', 'q', '4', 'E', 'F', 'H', 'L',
#           'W', 'w', 'V', 'v', 'U', 'X', 'x', 'Y', 'y', 'Z','z','7','K', 'R', 'k']

# L2Stage1=['6', 'e', 'r', 'P', 'p', 'A', 'M', 'N','m', 'n', 'h', 'b', 'a', 'u',
#           '0', 'O', 'o', 'D', 'Q','C', 'G', 'c', '3', 'B', '8', 'd', '2', '5', 's', 'S']

L1Stage1=['T', 'I', 'l', 'i', 't','j', '1', 'J', 'f', 'Z', 'z', '7', 'E', 'F', 'H', 'L', 'W', 'w', 'V', 'v', 'U', 'X', 'x', 'Y', 'y', 'P', 'p', 'A', 'K', 'R', 'k']
L2Stage1=['6', 'e', 'r', '5', 'S', 's','M', 'N','m', 'n', 'h', 'b', 'a', 'u', '0', 'O', 'o', 'D', 'Q','C', 'G', 'c', '3', 'B', '8','d','2', '9', 'g', 'q', '4']

stage1Train = ChangeLabel(dataTrain, L1Stage1 , 'Group1', L2Stage1, 'Group2')
stage1Eval  = ChangeLabel(dataTrain, L1Stage1 , 'Group1', L2Stage1, 'Group2')

y_trainS1 = stage1Train['label']
y_evalS1 = stage1Eval['label']

X_trainS1 = stage1Train.drop("label", axis = 1)
X_evalS1 = stage1Eval.drop("label", axis = 1)

SVM_stage1, params, score = SVM(X_trainS1, y_trainS1, 'SVM_stage1')

y_predS1 = SVM_stage1.predict(X_evalS1)

l = 'Best Params of SVM_stage1 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM_stage1 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM_stage1", metrics.accuracy_score(y_evalS1, y_predS1), "\n")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.696 total time=   4.2s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.870 total time=   5.3s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.826 total time=   5.5s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.796 total time=   5.1s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.719 total time=   4.5s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.542 total time=  14.3s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.500 total time=  14.1s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.625 total time=  13.4s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.501 total time=  13.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.517 total time=  14.0s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.678 total time=   3.9s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [ ]:
print('X_train shape:',X_trainS1.shape)
print('y_train shape:',y_trainS1.shape)
print('X_test shape:',X_evalS1.shape)
print('y_test shape:',y_evalS1.shape)

X_train shape: (2728, 822)
y_train shape: (2728,)
X_test shape: (2728, 822)
y_test shape: (2728,)


STAGE 2


In [ ]:
# L1Stage2=['T', 'I', 'l', 'i', 't','j', '1', 'J', 'f', '9', 'g', 'q', '4', 'E', 'F', 'H', 'L']

# L2Stage2=['W', 'w', 'V', 'v', 'U', 'X', 'x', 'Y', 'y', 'Z','z','7','K', 'R', 'k']

# L3Stage2=['6', 'e', 'r', 'P', 'p', 'A', 'M', 'N','m', 'n', 'h', 'b', 'a', 'u'] 

# L4Stage2=['0', 'O', 'o', 'D', 'Q','C', 'G', 'c', '3', 'B', '8', 'd', '2', '5', 's', 'S']

L1Stage2=['T', 'I', 'l', 'i', 't','j', '1', 'J', 'f', 'Z', 'z', '7', 'E', 'F', 'H', 'L']
L2Stage2=['W', 'w', 'V', 'v', 'U', 'X', 'x', 'Y', 'y', 'P', 'p', 'A', 'K', 'R', 'k']
L3Stage2=['6', 'e', 'r', '5', 'S', 's','M', 'N','m', 'n', 'h', 'b', 'a', 'u'] 
L4Stage2=['0', 'O', 'o', 'D', 'Q','C', 'G', 'c', '3', 'B', '8','d','2', '9', 'g', 'q', '4']

stage2Train1 = ChangeLabel(dataTrain, L1Stage2 , 'Group1', L2Stage2, 'Group2')
stage2Eval1  = ChangeLabel(dataTrain, L1Stage2 , 'Group1', L2Stage2, 'Group2')

y_train1S2 = stage2Train1['label']
y_eval1S2 = stage2Eval1['label']

X_train1S2 = stage2Train1.drop("label", axis = 1)
X_eval1S2 = stage2Eval1.drop("label", axis = 1)

SVM1_stage2, params, score = SVM(X_train1S2, y_train1S2, 'SVM1_stage2')

y_pred1S2 = SVM1_stage2.predict(X_eval1S2)
l = 'Best Params of SVM1_stage2 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM1_stage2 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM1_stage2", metrics.accuracy_score(y_eval1S2, y_pred1S2), "\n")


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.784 total time=   0.7s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.773 total time=   0.6s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.960 total time=   0.8s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.901 total time=   0.8s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.805 total time=   0.7s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.516 total time=   2.5s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.516 total time=   2.5s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.516 total time=   2.5s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.516 total time=   2.5s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.515 total time=   2.5s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.810 total time=   0.6s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [ ]:
print('X_train shape:',X_train1S2.shape)
print('y_train shape:',y_train1S2.shape)
print('X_test shape:',X_eval1S2.shape)
print('y_test shape:',y_eval1S2.shape)

In [76]:
stage2Train2 = ChangeLabel(dataTrain, L3Stage2 , 'Group1', L4Stage2, 'Group2')
stage2Eval2  = ChangeLabel(dataTrain, L3Stage2 , 'Group1', L4Stage2, 'Group2')

y_train2S2 = stage2Train2['label']
y_eval2S2 = stage2Eval2['label']

X_train2S2 = stage2Train2.drop("label", axis = 1)
X_eval2S2 = stage2Eval2.drop("label", axis = 1)

SVM2_stage2, params, score = SVM(X_train2S2, y_train2S2, 'SVM2_stage2')

y_pred2S2 = SVM2_stage2.predict(X_eval2S2)

l = 'Best Params of SVM2_stage2 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM2_stage2 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM2_stage2", metrics.accuracy_score(y_eval2S2, y_pred2S2), "\n")


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.758 total time=   1.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.707 total time=   1.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.678 total time=   1.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.755 total time=   1.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.816 total time=   1.1s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.546 total time=   2.3s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.549 total time=   2.4s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.549 total time=   2.5s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.549 total time=   2.5s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.548 total time=   2.4s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.656 total time=   0.8s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

STAGE 3

In [77]:
# L1Stage3=['T', 'I', 'l', 'i', 't','j', '1', 'J', 'f']
# L2Stage3=['9', 'g', 'q', '4', 'E', 'F', 'H', 'L']
# L3Stage3=['W', 'w', 'V', 'v', 'U', 'X', 'x', 'Y', 'y']  
# L4Stage3=['Z','z','7','K', 'R', 'k']
# L5Stage3=['6', 'e', 'r', 'P', 'p', 'A']
# L6Stage3=['M', 'N','m', 'n', 'h', 'b', 'a', 'u'] 
# L7Stage3=['0', 'O', 'o', 'D', 'Q','C', 'G', 'c']
# L8Stage3=['3', 'B', '8', 'd', '2', '5', 's', 'S'] 

L1Stage3=['T', 'I', 'l', 'i', 't','j', '1', 'J', 'f']
L2Stage3=['Z', 'z', '7', 'E', 'F', 'H', 'L']
L3Stage3=['W', 'w', 'V', 'v', 'U', 'X', 'x', 'Y', 'y'] 
L4Stage3=['P', 'p', 'A', 'K', 'R', 'k']
L5Stage3=['6', 'e', 'r', '5', 'S', 's']
L6Stage3=['M', 'N','m', 'n', 'h', 'b', 'a', 'u'] 
L7Stage3=['0', 'O', 'o', 'D', 'Q','C', 'G', 'c']
L8Stage3=['3', 'B', '8','d','2', '9', 'g', 'q', '4']



stage3Train1 = ChangeLabel(dataTrain, L1Stage3 , 'Group1', L2Stage3, 'Group2')
stage3Eval1  = ChangeLabel(dataTrain, L1Stage3 , 'Group1', L2Stage3, 'Group2')

y_train1S3 = stage3Train1['label']
y_eval1S3 = stage3Eval1['label']

X_train1S3 = stage3Train1.drop("label", axis = 1)
X_eval1S3 = stage3Eval1.drop("label", axis = 1)

SVM1_stage3 , params, score= SVM(X_train1S3, y_train1S3, 'SVM1_stage3')

y_pred1S3 = SVM1_stage3.predict(X_eval1S3)

l = 'Best Params of SVM1_stage3 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM1_stage3 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM1_stage3", metrics.accuracy_score(y_eval1S3, y_pred1S3), "\n")


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.801 total time=   0.2s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.943 total time=   0.2s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.929 total time=   0.2s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.922 total time=   0.2s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.943 total time=   0.2s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.567 total time=   0.7s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.560 total time=   0.7s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.560 total time=   0.7s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.560 total time=   0.7s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.564 total time=   0.7s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.837 total time=   0.1s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [78]:
stage3Train2 = ChangeLabel(dataTrain, L3Stage3 , 'Group1', L4Stage3, 'Group2')
stage3Eval2  = ChangeLabel(dataTrain, L3Stage3 , 'Group1', L4Stage3, 'Group2')

y_train2S3 = stage3Train2['label']
y_eval2S3 = stage3Eval2['label']

X_train2S3 = stage3Train2.drop("label", axis = 1)
X_eval2S3 = stage3Eval2.drop("label", axis = 1)

SVM2_stage3, params, score = SVM(X_train2S3, y_train2S3, 'SVM2_stage3')

y_pred2S3 = SVM2_stage3.predict(X_eval2S3)

l = 'Best Params of SVM2_stage3 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM2_stage3 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM2_stage3", metrics.accuracy_score(y_eval2S3, y_pred2S3), "\n")


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.864 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.992 total time=   0.1s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.992 total time=   0.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.985 total time=   0.1s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.917 total time=   0.1s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.606 total time=   0.6s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.598 total time=   0.6s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.598 total time=   0.6s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.598 total time=   0.6s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.598 total time=   0.6s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.924 total time=   0.1s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [79]:
stage3Train3 = ChangeLabel(dataTrain, L5Stage3 , 'Group1', L6Stage3, 'Group2')
stage3Eval3  = ChangeLabel(dataTrain, L5Stage3 , 'Group1', L6Stage3, 'Group2')

y_train3S3 = stage3Train3['label']
y_eval3S3 = stage3Eval3['label']

X_train3S3 = stage3Train3.drop("label", axis = 1)
X_eval3S3 = stage3Eval3.drop("label", axis = 1)

SVM3_stage3, params, score = SVM(X_train3S3, y_train3S3, 'SVM3_stage3')

y_pred3S3 = SVM3_stage3.predict(X_eval3S3)

l = 'Best Params of SVM3_stage3 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM3_stage3 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM3_stage3", metrics.accuracy_score(y_eval3S3, y_pred3S3), "\n")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.992 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.959 total time=   0.1s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.911 total time=   0.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.886 total time=   0.1s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.943 total time=   0.1s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.573 total time=   0.6s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.569 total time=   0.5s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.569 total time=   0.5s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.569 total time=   0.5s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.577 total time=   0.5s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.984 total time=   0.1s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [80]:
stage3Train4 = ChangeLabel(dataTrain, L7Stage3 , 'Group1', L8Stage3, 'Group2')
stage3Eval4  = ChangeLabel(dataTrain, L7Stage3 , 'Group1', L8Stage3, 'Group2')

y_train4S3 = stage3Train4['label']
y_eval4S3 = stage3Eval4['label']

# print(np.unique(stage3Train4['label']))
# print(np.unique(stage3Eval4['label']))

X_train4S3 = stage3Train4.drop("label", axis = 1)
X_eval4S3 = stage3Eval4.drop("label", axis = 1)

SVM4_stage3, params, score = SVM(X_train4S3, y_train4S3, 'SVM4_stage3')

y_pred4S3 = SVM4_stage3.predict(X_eval4S3)

l = 'Best Params of SVM4_stage3 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM4_stage3 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM4_stage3", metrics.accuracy_score(y_eval4S3, y_pred4S3), "\n")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.993 total time=   0.2s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.807 total time=   0.2s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.980 total time=   0.2s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.946 total time=   0.2s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.987 total time=   0.2s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.567 total time=   0.8s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.527 total time=   0.8s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.593 total time=   0.8s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time=   0.8s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.570 total time=   0.8s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.993 total time=   0.2s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

STAGE 4

In [81]:
L1Stage4=['T', 'I', 'l', 'i'] 
L2Stage4=['t','j', '1', 'J', 'f']
L3Stage4=['Z', 'z', '7'] 
L4Stage4=['E', 'F', 'H', 'L']
L5Stage4=['W', 'w', 'V', 'v', 'U']
L6Stage4=['X', 'x', 'Y', 'y'] 
L7Stage4=['P', 'p', 'A']
L8Stage4=['K', 'R', 'k']
L9Stage4=['6', 'e', 'r']
L10Stage4=['5', 'S', 's']
L11Stage4=['M', 'N','m', 'n'] 
L12Stage4=['h', 'b', 'a', 'u'] 
L13Stage4=['0', 'O', 'o', 'D']
L14Stage4=['Q','C', 'G', 'c']
L15Stage4=['3', 'B', '8','d','2'] 
L16Stage4=['9', 'g', 'q', '4']

# L1Stage4=['T', 'I', 'l', 'i'] 
# L2Stage4=['t','j', '1', 'J', 'f']
# L3Stage4=['9', 'g', 'q', '4'] 
# L4Stage4=['E', 'F', 'H', 'L']
# L5Stage4=['W', 'w', 'V', 'v', 'U']
# L6Stage4=['X', 'x', 'Y', 'y'] 
# L7Stage4=['Z','z','7']
# L8Stage4=['K', 'R', 'k']
# L9Stage4=['6', 'e', 'r']
# L10Stage4=['P', 'p', 'A']
# L11Stage4=['M', 'N','m', 'n'] 
# L12Stage4=['h', 'b', 'a', 'u'] 
# L13Stage4=['0', 'O', 'o', 'D']
# L14Stage4=['Q','C', 'G', 'c']
# L15Stage4=['3', 'B', '8', 'd', '2'] 
# L16Stage4=['5', 's', 'S'] 

stage4Train1 = ChangeLabel(dataTrain, L1Stage4 , 'Group1', L2Stage4, 'Group2')
stage4Eval1  = ChangeLabel(dataTrain, L1Stage4 , 'Group1', L2Stage4, 'Group2')
stage4Train2 = ChangeLabel(dataTrain, L3Stage4 , 'Group1', L4Stage4, 'Group2')
stage4Eval2  = ChangeLabel(dataTrain, L3Stage4 , 'Group1', L4Stage4, 'Group2')
stage4Train3 = ChangeLabel(dataTrain, L5Stage4 , 'Group1', L6Stage4, 'Group2')
stage4Eval3  = ChangeLabel(dataTrain, L5Stage4 , 'Group1', L6Stage4, 'Group2')
stage4Train4 = ChangeLabel(dataTrain, L7Stage4 , 'Group1', L8Stage4, 'Group2')
stage4Eval4  = ChangeLabel(dataTrain, L7Stage4 , 'Group1', L8Stage4, 'Group2')
stage4Train5 = ChangeLabel(dataTrain, L9Stage4 , 'Group1', L10Stage4, 'Group2')
stage4Eval5  = ChangeLabel(dataTrain, L9Stage4 , 'Group1', L10Stage4, 'Group2')
stage4Train6 = ChangeLabel(dataTrain, L11Stage4 , 'Group1', L12Stage4, 'Group2')
stage4Eval6  = ChangeLabel(dataTrain, L11Stage4 , 'Group1', L12Stage4, 'Group2')
stage4Train7 = ChangeLabel(dataTrain, L13Stage4 , 'Group1', L14Stage4, 'Group2')
stage4Eval7  = ChangeLabel(dataTrain, L13Stage4 , 'Group1', L14Stage4, 'Group2')
stage4Train8 = ChangeLabel(dataTrain, L15Stage4 , 'Group1', L16Stage4, 'Group2')
stage4Eval8  = ChangeLabel(dataTrain, L1Stage4 , 'Group1', L16Stage4, 'Group2')

y_train1S4 = stage4Train1['label']
y_eval1S4 = stage4Eval1['label']
y_train2S4 = stage4Train2['label']
y_eval2S4 = stage4Eval2['label']
y_train3S4 = stage4Train3['label']
y_eval3S4 = stage4Eval3['label']
y_train4S4 = stage4Train4['label']
y_eval4S4 = stage4Eval4['label']
y_train5S4 = stage4Train5['label']
y_eval5S4 = stage4Eval5['label']
y_train6S4 = stage4Train6['label']
y_eval6S4 = stage4Eval6['label']
y_train7S4 = stage4Train7['label']
y_eval7S4 = stage4Eval7['label']
y_train8S4 = stage4Train8['label']
y_eval8S4 = stage4Eval8['label']


X_train1S4 = stage4Train1.drop("label", axis = 1)
X_eval1S4 = stage4Eval1.drop("label", axis = 1)
X_train2S4 = stage4Train2.drop("label", axis = 1)
X_eval2S4 = stage4Eval2.drop("label", axis = 1)
X_train3S4 = stage4Train3.drop("label", axis = 1)
X_eval3S4 = stage4Eval3.drop("label", axis = 1)
X_train4S4 = stage4Train4.drop("label", axis = 1)
X_eval4S4 = stage4Eval4.drop("label", axis = 1)
X_train5S4 = stage4Train5.drop("label", axis = 1)
X_eval5S4 = stage4Eval5.drop("label", axis = 1)
X_train6S4 = stage4Train6.drop("label", axis = 1)
X_eval6S4 = stage4Eval6.drop("label", axis = 1)
X_train7S4 = stage4Train7.drop("label", axis = 1)
X_eval7S4 = stage4Eval7.drop("label", axis = 1)
X_train8S4 = stage4Train8.drop("label", axis = 1)
X_eval8S4 = stage4Eval8.drop("label", axis = 1)

8 Models of STAGE 4

In [82]:
SVM1_stage4, params, score = SVM(X_train1S4, y_train1S4, 'SVM1_stage4')

y_pred1S4 = SVM1_stage4.predict(X_eval1S4)

l = 'Best Params of SVM1_stage4 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM1_stage4 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM1_stage4", metrics.accuracy_score(y_eval1S4, y_pred1S4), "\n")


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.450 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.646 total time=   0.1s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.911 total time=   0.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.975 total time=   0.1s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.658 total time=   0.1s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.550 total time=   0.2s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time=   0.3s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time=   0.2s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time=   0.2s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time=   0.3s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.438 total time=   0.1s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [83]:
print(metrics.confusion_matrix(y_true=y_eval1S4, y_pred=y_pred1S4))

[[173   3]
 [ 16 204]]


In [84]:
SVM2_stage4, params, score = SVM(X_train2S4, y_train2S4, 'SVM2_stage4')

y_pred2S4 = SVM2_stage4.predict(X_eval2S4)

l = 'Best Params of SVM2_stage4 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM2_stage4 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM2_stage4", metrics.accuracy_score(y_eval2S4, y_pred2S4), "\n")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.968 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.968 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.968 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.967 total time=   0.1s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.565 total time=   0.1s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.565 total time=   0.2s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.581 total time=   0.2s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.574 total time=   0.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.574 total time=   0.1s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.984 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [85]:
SVM3_stage4, params, score = SVM(X_train3S4, y_train3S4, 'SVM3_stage4')

y_pred3S4 = SVM3_stage4.predict(X_eval3S4)

l = 'Best Params of SVM3_stage4 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM3_stage4 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM3_stage4", metrics.accuracy_score(y_eval3S4, y_pred3S4), "\n")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.1s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.962 total time=   0.1s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.1s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.550 total time=   0.2s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time=   0.2s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time=   0.2s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time=   0.2s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time=   0.2s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=1.000 total time=   0.1s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [86]:
SVM4_stage4 , params, score = SVM(X_train4S4, y_train4S4, 'SVM4_stage4')

y_pred4S4 = SVM4_stage4.predict(X_eval4S4)

l = 'Best Params of SVM4_stage4 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM4_stage4 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM4_stage4", metrics.accuracy_score(y_eval4S4, y_pred4S4), "\n")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.981 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.981 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.962 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.981 total time=   0.0s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.491 total time=   0.1s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.491 total time=   0.1s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.491 total time=   0.1s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.491 total time=   0.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.673 total time=   0.1s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=1.000 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [87]:
SVM5_stage4, params, score = SVM(X_train5S4, y_train5S4, 'SVM5_stage4')

y_pred5S4 = SVM5_stage4.predict(X_eval5S4)

l = 'Best Params of SVM5_stage4 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM5_stage4 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM5_stage4", metrics.accuracy_score(y_eval5S4, y_pred5S4), "\n")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.981 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.1s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.868 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.942 total time=   0.1s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.491 total time=   0.1s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.491 total time=   0.1s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.491 total time=   0.1s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.491 total time=   0.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.673 total time=   0.1s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.981 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [88]:
SVM6_stage4, params, score = SVM(X_train6S4, y_train6S4, 'SVM6_stage4')

y_pred6S4 = SVM6_stage4.predict(X_eval6S4)

l = 'Best Params of SVM6_stage4 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM6_stage4 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM6_stage4", metrics.accuracy_score(y_eval6S4, y_pred6S4), "\n")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.986 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.972 total time=   0.1s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.971 total time=   0.1s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.943 total time=   0.1s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.493 total time=   0.2s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.493 total time=   0.2s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.800 total time=   0.2s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.514 total time=   0.2s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.500 total time=   0.2s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.972 total time=   0.1s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [89]:
SVM7_stage4, params, score = SVM(X_train7S4, y_train7S4, 'SVM7_stage4')

y_pred7S4 = SVM7_stage4.predict(X_eval7S4)

l = 'Best Params of SVM7_stage4 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM7_stage4 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM7_stage4", metrics.accuracy_score(y_eval7S4, y_pred7S4), "\n")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.1s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.986 total time=   0.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.943 total time=   0.1s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=1.000 total time=   0.1s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.606 total time=   0.2s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.606 total time=   0.2s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.800 total time=   0.2s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.886 total time=   0.2s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.671 total time=   0.2s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=1.000 total time=   0.1s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [90]:
SVM8_stage4,params, score = SVM(X_train8S4, y_train8S4, 'SVM8_stage4')

y_pred8S4 = SVM8_stage4.predict(X_eval8S4)

l = 'Best Params of SVM8_stage4 with White: '+str(params)+'\n'
l2 = 'Best training accuracy of SVM8_stage4 with White: '+str(score)+'\n'
print(l)
print(l2)
print("evaluation accuracy of SVM8_stage4", metrics.accuracy_score(y_eval8S4, y_pred8S4), "\n")
print(metrics.confusion_matrix(y_true=y_eval8S4, y_pred=y_pred8S4))

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.988 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.873 total time=   0.1s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.772 total time=   0.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.861 total time=   0.1s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.937 total time=   0.1s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.550 total time=   0.2s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time=   0.2s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time=   0.3s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time=   0.2s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time=   0.2s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.912 total time=   0.1s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [91]:
def getFinalStageData(L):
  data1 = pd.DataFrame()
  data1 = data1.append(dataTrain)
  ndata1= data1.loc[data1['label'].isin(L)]

  data2 = pd.DataFrame()
  data2 = data1.append(dataEval)
  ndata2= data2.loc[data2['label'].isin(L)] 

  y_train = ndata1['label']
  X_train = ndata1.drop("label", axis = 1)

  y_eval = ndata2['label']
  X_eval = ndata2.drop("label", axis = 1)

  return X_train, y_train, X_eval, y_eval

In [117]:
# L1=['T', 'I', 'l', 'i'] 
# L2=['t','j', '1', 'J', 'f']
# L3=['9', 'g', 'q', '4'] 
# L4=['E', 'F', 'H', 'L']
# L5=['W', 'w', 'V', 'v', 'U']
# L6=['X', 'x', 'Y', 'y'] 
# L7=['Z','z','7']
# L8=['K', 'R', 'k']
# L9=['6', 'e', 'r']
# L10=['P', 'p', 'A']
# L11=['M', 'N','m', 'n'] 
# L12=['h', 'b', 'a', 'u'] 
# L13=['0', 'O', 'o', 'D']
# L14=['Q','C', 'G', 'c']
# L15=['3', 'B', '8', 'd', '2'] 
# L16=['5', 's', 'S'] 
# Lists = [L1, L2, L3, L4, L5, L6, L7, L8, L9, L10, L11, L12, L13, L14, L15, L16]
# LL8 = ['3', 'B', '8','d', '9', 'g', 'q', '4', '2']

L1=['T', 'I', 'l', 'i'] 
L2=['t','j', '1', 'J', 'f']
L3= ['Z','z','7']
L4=['E', 'F', 'H', 'L']
L5=['W', 'w', 'V', 'v', 'U']
L6=['X', 'x', 'Y', 'y'] 
L7= ['P', 'p', 'A']
L8=['K', 'R', 'k']
L9=['6', 'e', 'r']
L10=['5', 's', 'S'] 
L11=['M', 'N','m', 'n'] 
L12=['h', 'b', 'a', 'u'] 
L13=['0', 'O', 'o', 'D']
L14=['Q','C', 'G', 'c']
L15=['3', 'B', '8', 'd', '2'] 
L16= ['9', 'g', 'q', '4'] 
Lists = [L1, L2, L3, L4, L5, L6, L7, L8, L9, L10, L11, L12, L13, L14, L15, L16]
LL8 = ['3', 'B', '8', 'd', '2', '9', 'g', 'q', '4'] 

In [118]:
i = 1
for L in Lists:
  X_train, y_train, X_eval, y_eval = getFinalStageData(L)
  tempSVC, params, score = SVM(X_train, y_train, f'FinalStageSVM{i}')
  y_pred = tempSVC.predict(X_eval)

  print(f'Best Params of FinalStageSVM{i} with White: '+str(params)+'\n')
  print(f'Best training accuracy of FinalStageSVM{i} with White: '+str(score)+'\n')

  print(f"accuracy of FinalStageSVM{i}", metrics.accuracy_score(y_eval, y_pred), "\n")
  #print(metrics.confusion_matrix(y_true=y_eval, y_pred=y_pred))
  i=i+1

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.889 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.829 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.829 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.943 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.886 total time=   0.0s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.500 total time=   0.1s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.229 total time=   0.1s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.229 total time=   0.1s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.229 total time=   0.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.229 total time=   0.1s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.806 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [108]:
X_train, y_train, X_eval, y_eval = getFinalStageData(L16)
tempSVC, params, score = SVM(X_train, y_train, f'FinalStageSVM16')
y_pred = tempSVC.predict(X_eval)

print(f'Best Params of FinalStageSVM16 with White: '+str(params)+'\n')
print(f'Best training accuracy of FinalStageSVM16 with White: '+str(score)+'\n')

print(f"accuracy of FinalStageSVM16", metrics.accuracy_score(y_eval, y_pred), "\n")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.741 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.889 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.808 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.808 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.769 total time=   0.0s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.481 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.667 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.308 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.308 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.308 total time=   0.0s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.741 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [119]:


X_train, y_train, X_eval, y_eval = getFinalStageData(LL8)

SVM8_Stage_4_Final_combined , params, score= SVM(X_train, y_train, 'SVM8_Stage_4_Final_combined')

y_pred = SVM8_Stage_4_Final_combined.predict(X_eval)

print('Best Params of SVM8_Stage_4_Final_combined with White: '+str(params)+'\n')
print('Best training accuracy of SVM8_Stage_4_Final_combined with White: '+str(score)+'\n')
print("accuracy of SVM8_Stage_4_Final_combined", metrics.accuracy_score(y_eval, y_pred), "\n")
print(metrics.confusion_matrix(y_true=y_eval, y_pred=y_pred))

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.900 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.899 total time=   0.1s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.873 total time=   0.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.848 total time=   0.1s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.797 total time=   0.1s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.100 total time=   0.3s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.152 total time=   0.3s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.127 total time=   0.3s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.101 total time=   0.3s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.177 total time=   0.3s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.850 total time=   0.1s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

In [120]:
def Top3(predictions_proba):

  predictions_proba = predictions_proba.tolist()
  lea = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
  predictions=[]

  index1 = predictions_proba.index(max(predictions_proba))
  predictions.append(lea[index1])
  predictions.append(predictions_proba[index1]) 
  predictions_proba[index1] = 0

  index2 = predictions_proba.index(max(predictions_proba))
  predictions.append(lea[index2])
  predictions.append(predictions_proba[index2]) 
  predictions_proba[index2] = 0

  index3 = predictions_proba.index(max(predictions_proba))
  predictions.append(lea[index3])
  predictions.append(predictions_proba[index3]) 
  predictions_proba[index3] = 0
 
  return predictions_proba, predictions

Building the Sequential Ensemble

In [121]:
SVM_stage1 = joblib.load('SVM_stage1.npy')

SVM1_stage2 = joblib.load('SVM1_stage2.npy')
SVM2_stage2 = joblib.load('SVM2_stage2.npy')

SVM1_stage3 = joblib.load('SVM1_stage3.npy')
SVM2_stage3 = joblib.load('SVM2_stage3.npy')
SVM3_stage3 = joblib.load('SVM3_stage3.npy')
SVM4_stage3 = joblib.load('SVM4_stage3.npy')

SVM1_stage4 = joblib.load('SVM1_stage4.npy')
SVM2_stage4 = joblib.load('SVM2_stage4.npy')
SVM3_stage4 = joblib.load('SVM3_stage4.npy')
SVM4_stage4 = joblib.load('SVM4_stage4.npy')
SVM5_stage4 = joblib.load('SVM5_stage4.npy')
SVM6_stage4 = joblib.load('SVM6_stage4.npy')
SVM7_stage4 = joblib.load('SVM7_stage4.npy')
SVM8_stage4 = joblib.load('SVM8_stage4.npy')

FinalStageSVM1 = joblib.load('FinalStageSVM1.npy')
FinalStageSVM2 = joblib.load('FinalStageSVM2.npy')
FinalStageSVM3 = joblib.load('FinalStageSVM3.npy')
FinalStageSVM4 = joblib.load('FinalStageSVM4.npy')
FinalStageSVM5 = joblib.load('FinalStageSVM5.npy')
FinalStageSVM6 = joblib.load('FinalStageSVM6.npy')
FinalStageSVM7 = joblib.load('FinalStageSVM7.npy')
FinalStageSVM8 = joblib.load('FinalStageSVM8.npy')
FinalStageSVM9 = joblib.load('FinalStageSVM9.npy')
FinalStageSVM10 = joblib.load('FinalStageSVM10.npy')
FinalStageSVM11 = joblib.load('FinalStageSVM11.npy')
FinalStageSVM12 = joblib.load('FinalStageSVM12.npy')
FinalStageSVM13 = joblib.load('FinalStageSVM13.npy')
FinalStageSVM14 = joblib.load('FinalStageSVM14.npy')
FinalStageSVM15 = joblib.load('FinalStageSVM15.npy')
FinalStageSVM16 = joblib.load('FinalStageSVM16.npy')

def SequentialModel2(input):
  y_stage1_proba = SVM_stage1.predict_proba(input)
  y_stage1_proba = y_stage1_proba.tolist()
  y_stage1_p = max(y_stage1_proba)
  y_stage1 = SVM_stage1.predict(input)


  #Stage 1
  if y_stage1 == 'Group1':
    y_stage2 = SVM1_stage2.predict(input)
    y_stage2_proba = SVM1_stage2.predict_proba(input)
    y_stage2_proba = y_stage2_proba.tolist()
    y_stage2_p = max(y_stage2_proba)

    #Stage 2
    if y_stage2 == 'Group1':
      y_stage3 = SVM1_stage3.predict(input)

      #Stage 3
      if y_stage3 == 'Group1':
        y_stage4 = SVM1_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM1.predict(input)
        else:
          #Final
          output = FinalStageSVM2.predict(input)
      else:
        y_stage4 = SVM2_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM3.predict(input)
        else:
          #Final
          output = FinalStageSVM4.predict(input)
    else:
      y_stage3 = SVM2_stage3.predict(input)

      #Stage 3
      if y_stage3 == 'Group1':
        y_stage4 = SVM3_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM5.predict(input)
        else:
          #Final
          output = FinalStageSVM6.predict(input)
      else:
        y_stage4 = SVM4_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM7.predict(input)
        else:
          #Final
          output = FinalStageSVM8.predict(input)
  
  else:
    y_stage2 = SVM2_stage2.predict(input)

    #Stage 2
    if y_stage2 == 'Group1':
      y_stage3 = SVM3_stage3.predict(input)

      #Stage 3
      if y_stage3 == 'Group1':
        y_stage4 = SVM5_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM9.predict(input)
        else:
          #Final
          output = FinalStageSVM10.predict(input)
      else:
        y_stage4 = SVM6_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM11.predict(input)
        else:
          #Final
          output = FinalStageSVM12.predict(input)
    else:
      y_stage3 = SVM4_stage3.predict(input)

      #Stage 3
      if y_stage3 == 'Group1':
        y_stage4 = SVM7_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM13.predict(input)
        else:
          #Final
          output = FinalStageSVM14.predict(input)
      else:
        y_stage4 = SVM8_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM15.predict(input)
        else:
          #Final
          output = FinalStageSVM16.predict(input)
  return output

In [122]:
SVM_stage1 = joblib.load('SVM_stage1.npy')

SVM1_stage2 = joblib.load('SVM1_stage2.npy')
SVM2_stage2 = joblib.load('SVM2_stage2.npy')

SVM1_stage3 = joblib.load('SVM1_stage3.npy')
SVM2_stage3 = joblib.load('SVM2_stage3.npy')
SVM3_stage3 = joblib.load('SVM3_stage3.npy')
SVM4_stage3 = joblib.load('SVM4_stage3.npy')

SVM1_stage4 = joblib.load('SVM1_stage4.npy')
SVM2_stage4 = joblib.load('SVM2_stage4.npy')
SVM3_stage4 = joblib.load('SVM3_stage4.npy')
SVM4_stage4 = joblib.load('SVM4_stage4.npy')
SVM5_stage4 = joblib.load('SVM5_stage4.npy')
SVM6_stage4 = joblib.load('SVM6_stage4.npy')
SVM7_stage4 = joblib.load('SVM7_stage4.npy')
SVM8_stage4 = joblib.load('SVM8_stage4.npy')

FinalStageSVM1 = joblib.load('FinalStageSVM1.npy')
FinalStageSVM2 = joblib.load('FinalStageSVM2.npy')
FinalStageSVM3 = joblib.load('FinalStageSVM3.npy')
FinalStageSVM4 = joblib.load('FinalStageSVM4.npy')
FinalStageSVM5 = joblib.load('FinalStageSVM5.npy')
FinalStageSVM6 = joblib.load('FinalStageSVM6.npy')
FinalStageSVM7 = joblib.load('FinalStageSVM7.npy')
FinalStageSVM8 = joblib.load('FinalStageSVM8.npy')
FinalStageSVM9 = joblib.load('FinalStageSVM9.npy')
FinalStageSVM10 = joblib.load('FinalStageSVM10.npy')
FinalStageSVM11 = joblib.load('FinalStageSVM11.npy')
FinalStageSVM12 = joblib.load('FinalStageSVM12.npy')
FinalStageSVM13 = joblib.load('FinalStageSVM13.npy')
FinalStageSVM14 = joblib.load('FinalStageSVM14.npy')
FinalStageSVM15 = joblib.load('FinalStageSVM15.npy')
FinalStageSVM16 = joblib.load('FinalStageSVM16.npy')

SVM8_Stage_4_Final_combined= joblib.load('SVM8_Stage_4_Final_combined.npy')
 
def SequentialModel(input):
  y_stage1 = SVM_stage1.predict(input)

  #Stage 1
  if y_stage1 == 'Group1':
    y_stage2 = SVM1_stage2.predict(input)

    #Stage 2
    if y_stage2 == 'Group1':
      y_stage3 = SVM1_stage3.predict(input)

      #Stage 3
      if y_stage3 == 'Group1':
        y_stage4 = SVM1_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM1.predict(input)
        else:
          #Final
          output = FinalStageSVM2.predict(input)
      else:
        y_stage4 = SVM2_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM3.predict(input)
        else:
          #Final
          output = FinalStageSVM4.predict(input)
    else:
      y_stage3 = SVM2_stage3.predict(input)

      #Stage 3
      if y_stage3 == 'Group1':
        y_stage4 = SVM3_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM5.predict(input)
        else:
          #Final
          output = FinalStageSVM6.predict(input)
      else:
        y_stage4 = SVM4_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM7.predict(input)
        else:
          #Final
          output = FinalStageSVM8.predict(input)
  
  else:
    y_stage2 = SVM2_stage2.predict(input)

    #Stage 2
    if y_stage2 == 'Group1':
      y_stage3 = SVM3_stage3.predict(input)

      #Stage 3
      if y_stage3 == 'Group1':
        y_stage4 = SVM5_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM9.predict(input)
        else:
          #Final
          output = FinalStageSVM10.predict(input)
      else:
        y_stage4 = SVM6_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM11.predict(input)
        else:
          #Final
          output = FinalStageSVM12.predict(input)
    else:
      y_stage3 = SVM4_stage3.predict(input)

      #Stage 3
      if y_stage3 == 'Group1':
        y_stage4 = SVM7_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM13.predict(input)
        else:
          #Final
          output = FinalStageSVM14.predict(input)
      else:
        output = SVM8_Stage_4_Final_combined.predict(input)
  return output

In [123]:
# LL1=['T', 'I', 'l', 'i', 't','j', '1', 'J']

# X_train, y_train, X_eval, y_eval = getFinalStageData(LL1)

# SVM1_Stage_4_Final_combined , params, score= SVM(X_train, y_train, 'SVM1_Stage_4_Final_combined')

# y_pred = SVM1_Stage_4_Final_combined.predict(X_eval)

# print('Best Params of SVM1_Stage_4_Final_combined with White: '+str(params)+'\n')
# print('Best training accuracy of SVM1_Stage_4_Final_combined with White: '+str(score)+'\n')
# print("accuracy of SVM1_Stage_4_Final_combined", metrics.accuracy_score(y_eval, y_pred), "\n")
# print(metrics.confusion_matrix(y_true=y_eval, y_pred=y_pred))

In [124]:
SVM_stage1 = joblib.load('SVM_stage1.npy')

SVM1_stage2 = joblib.load('SVM1_stage2.npy')
SVM2_stage2 = joblib.load('SVM2_stage2.npy')

SVM1_stage3 = joblib.load('SVM1_stage3.npy')
SVM2_stage3 = joblib.load('SVM2_stage3.npy')
SVM3_stage3 = joblib.load('SVM3_stage3.npy')
SVM4_stage3 = joblib.load('SVM4_stage3.npy')

SVM1_stage4 = joblib.load('SVM1_stage4.npy')
SVM2_stage4 = joblib.load('SVM2_stage4.npy')
SVM3_stage4 = joblib.load('SVM3_stage4.npy')
SVM4_stage4 = joblib.load('SVM4_stage4.npy')
SVM5_stage4 = joblib.load('SVM5_stage4.npy')
SVM6_stage4 = joblib.load('SVM6_stage4.npy')
SVM7_stage4 = joblib.load('SVM7_stage4.npy')
SVM8_stage4 = joblib.load('SVM8_stage4.npy')

FinalStageSVM1 = joblib.load('FinalStageSVM1.npy')
FinalStageSVM2 = joblib.load('FinalStageSVM2.npy')
FinalStageSVM3 = joblib.load('FinalStageSVM3.npy')
FinalStageSVM4 = joblib.load('FinalStageSVM4.npy')
FinalStageSVM5 = joblib.load('FinalStageSVM5.npy')
FinalStageSVM6 = joblib.load('FinalStageSVM6.npy')
FinalStageSVM7 = joblib.load('FinalStageSVM7.npy')
FinalStageSVM8 = joblib.load('FinalStageSVM8.npy')
FinalStageSVM9 = joblib.load('FinalStageSVM9.npy')
FinalStageSVM10 = joblib.load('FinalStageSVM10.npy')
FinalStageSVM11 = joblib.load('FinalStageSVM11.npy')
FinalStageSVM12 = joblib.load('FinalStageSVM12.npy')
FinalStageSVM13 = joblib.load('FinalStageSVM13.npy')
FinalStageSVM14 = joblib.load('FinalStageSVM14.npy')
FinalStageSVM15 = joblib.load('FinalStageSVM15.npy')
FinalStageSVM16 = joblib.load('FinalStageSVM16.npy')

SVM8_Stage_4_Final_combined= joblib.load('SVM8_Stage_4_Final_combined.npy')
 
def SequentialModel3(input):
  y_stage1 = SVM_stage1.predict(input)

  #Stage 1
  if y_stage1 == 'Group1':
    y_stage2 = SVM1_stage2.predict(input)

    #Stage 2
    if y_stage2 == 'Group1':
      y_stage3 = SVM1_stage3.predict(input)

      #Stage 3
      if y_stage3 == 'Group1':
        output = SVM1_Stage_4_Final_combined.predict(input)

        #Stage 4
        # if y_stage4 == 'Group1':
        #   #Final stage
        #   output = FinalStageSVM1.predict(input)
        # else:
        #   #Final
        #   output = FinalStageSVM2.predict(input)
      else:
        y_stage4 = SVM2_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM3.predict(input)
        else:
          #Final
          output = FinalStageSVM4.predict(input)
    else:
      y_stage3 = SVM2_stage3.predict(input)

      #Stage 3
      if y_stage3 == 'Group1':
        y_stage4 = SVM3_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM5.predict(input)
        else:
          #Final
          output = FinalStageSVM6.predict(input)
      else:
        y_stage4 = SVM4_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM7.predict(input)
        else:
          #Final
          output = FinalStageSVM8.predict(input)
  
  else:
    y_stage2 = SVM2_stage2.predict(input)

    #Stage 2
    if y_stage2 == 'Group1':
      y_stage3 = SVM3_stage3.predict(input)

      #Stage 3
      if y_stage3 == 'Group1':
        y_stage4 = SVM5_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM9.predict(input)
        else:
          #Final
          output = FinalStageSVM10.predict(input)
      else:
        y_stage4 = SVM6_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM11.predict(input)
        else:
          #Final
          output = FinalStageSVM12.predict(input)
    else:
      y_stage3 = SVM4_stage3.predict(input)

      #Stage 3
      if y_stage3 == 'Group1':
        y_stage4 = SVM7_stage4.predict(input)

        #Stage 4
        if y_stage4 == 'Group1':

          #Final stage
          output = FinalStageSVM13.predict(input)
        else:
          #Final
          output = FinalStageSVM14.predict(input)
      else:
        output = SVM8_Stage_4_Final_combined.predict(input)
  return output

In [128]:
!zip -r /content/FinalEnsemble1.zip /content/

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2022.10.28/ (stored 0%)
  adding: content/.config/logs/2022.10.28/13.35.23.262379.log (deflated 91%)
  adding: content/.config/logs/2022.10.28/13.37.01.262813.log (deflated 53%)
  adding: content/.config/logs/2022.10.28/13.36.19.156679.log (deflated 86%)
  adding: content/.config/logs/2022.10.28/13.37.00.362876.log (deflated 54%)
  adding: content/.config/logs/2022.10.28/13.35.53.106765.log (deflated 53%)
  adding: content/.config/logs/2022.10.28/13.36.34.707746.log (deflated 53%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 23%)
  adding: content/.config/.last

In [125]:
def getYEvaluation ():
    df = pd.read_csv('EvalProfileWhite.csv')
    image_names = df.iloc[:,0]
    image_names = image_names.values.tolist()
    print(len(image_names))

    y_output = []

    for image in image_names:
        df_englishCSV = pd.read_csv('english.csv')
        label = df_englishCSV.loc[df_englishCSV['image'] == image]
        label = label.drop(['image'], axis=1)
        label = label.values.tolist()
        y_output.append(label[0][0])

    return y_output


def getFeatureVector():
    df = pd.read_csv('EvalProfileWhite.csv')
    df = df.drop(['image', 'label'], axis=1)
    feature_vectors = df.values.tolist()
    return feature_vectors

In [126]:
import warnings
warnings.filterwarnings("ignore")
predicted_output = []
feature_Vector = getFeatureVector()
actual_output = getYEvaluation()

for idx, image in enumerate(feature_Vector):
    feature = []
    feature.append(image)
    predicted_y = SequentialModel2(feature)
    predicted_output.append(predicted_y)

    print(f'predicted output : {predicted_y} | {actual_output[idx]} :Actual output')

print(actual_output)
print(len(actual_output))
print(predicted_output)
print(len(predicted_output))

682
predicted output : ['0'] | 0 :Actual output
predicted output : ['O'] | 0 :Actual output
predicted output : ['0'] | 0 :Actual output
predicted output : ['O'] | 0 :Actual output
predicted output : ['O'] | 0 :Actual output
predicted output : ['O'] | 0 :Actual output
predicted output : ['0'] | 0 :Actual output
predicted output : ['o'] | 0 :Actual output
predicted output : ['e'] | 0 :Actual output
predicted output : ['0'] | 0 :Actual output
predicted output : ['O'] | 0 :Actual output
predicted output : ['1'] | 1 :Actual output
predicted output : ['2'] | 1 :Actual output
predicted output : ['1'] | 1 :Actual output
predicted output : ['1'] | 1 :Actual output
predicted output : ['l'] | 1 :Actual output
predicted output : ['I'] | 1 :Actual output
predicted output : ['I'] | 1 :Actual output
predicted output : ['9'] | 1 :Actual output
predicted output : ['l'] | 1 :Actual output
predicted output : ['I'] | 1 :Actual output
predicted output : ['1'] | 1 :Actual output
predicted output : ['2'] | 2

In [130]:
# print(metrics.confusion_matrix(actual_output, predicted_output), "\n")
print("accuracy", metrics.accuracy_score(actual_output, predicted_output), "\n")

accuracy 0.8225806451612904 

